<a href="https://colab.research.google.com/github/drusho/bigquery_liquor_sales/blob/main/notebooks/2021_08_01_bigquery_liquore_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using SQL to Analyze Iowa Liquor Sales
> Exploring Public Datasets in Google BigQuery


Tags
* bigquery, pandas, plotly, sales, sql





## About the Data 

__Data Source:__ [data.iowa.gov](https://data.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy)

__Provided by:__ Iowa Department of Commerce, Alcoholic Beverages Division

__Data Created:__ November 7, 2014

__Last Updated:__ July 1, 2021

This dataset contains the spirits purchase information of Iowa Class “E” liquor licensees by product and date of purchase from January 1, 2014 to current. The dataset can be used to analyze total spirits sales in Iowa of individual products at the store level.

Class E liquor license, for grocery stores, liquor stores, convenience stores, etc., allows commercial establishments to sell liquor for off-premises consumption in original unopened containers.

### Goals
Determine sales trends for city/counties
* Popular items sold
* Locations for highest sales
* Dates of most sales

### Column Descriptions [(source)](https://data.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy)

<br> 

| Column Name | Description | Data Type |
|:--|:--|:--|
| Invoice/Item Number | Concatenated invoice and line number associated with the liquor order. | Plain Text |
| Date | Date of Order | Date/Time |
| Store Number | Unique number assigned to the store who ordered the liquor. | Plain Text |
| Store Name | Name of store who ordered the liquor. | Plain Text  |
| City | City where the store who ordered the liquor is located | Plain Text  |
| Zip Code | Zip Code where the store who ordered the liquor is located | Plain Text  |
| Store Location | Location of store who ordered the liquor. | Plain Text  |
| County Number | Iowa county number for the county where store who ordered the liquor is located | Plain Text  |
| County | County where the store who ordered the liquor is located | Plain Text  |
| Category | Category code associated with the liquor ordered | Plain Text  |
| Category Name | Category of the liquor ordered | Plain Text  |
| Vendor Number | The vendor number of the company for the brand of liquor ordered | Plain Text  |
| Vendor Name | The vendor name of the company for the brand of liquor ordered | Plain Text  |
| Item Number | Item number for the individual liquor product ordered. | Plain Text  |
| Item Description | Description of the individual liquor product ordered. | Plain Text  |
| Pack | The number of bottles in a case for the liquor ordered | Number  |
| Bottle Volume (ml) | Volume of each liquor bottle ordered in milliliters. | Number  |
| State Bottle Cost | The amount that Alcoholic Beverages Division paid for each bottle of liquor ordered | Number  |
| State Bottle Retail | The amount the store paid for each bottle of liquor ordered | Number |
| Bottles Sold | The amount the store paid for each bottle of liquor ordered | Number |
| Sale (Dollars) | Total cost of liquor order (number of bottles multiplied by the state bottle retail) | Number |
| Volume Sold (Liters) | Total volume of liquor ordered in liters. (i.e. (Bottle Volume (ml) x Bottles Sold)/1,000) | Number |
| Volume Sold (Gallons | Total volume of liquor ordered in gallons. (i.e. (Bottle Volume (ml) x Bottles Sold)/3785.411784) | Number |

<br>

_*Store Location_: 

The Address, City, State and Zip Code are geocoded to provide geographic coordinates. Accuracy of geocoding is dependent on how well the address is interpreted and the completeness of the reference data used. 




### Todo 

__date__
* [x] Date of last update to dataset (return 1 item only)
* [x] day of the week w/ most sales
* [x] Month w/ most sales
* [X] holiday(s) w/ most sales

__sale_dollars/city/county__
* [x] County/City with most liquor sales (top 5)

__sale_dollars/vendor_name__
* [x] most sales

__item_description/sale_dollars/
* [x] top 5 sales

__store_location__
* [ ] map coordinatos by most sales
 

In [ ]:
# must upgrade plotly to avoid duplicate legend labels 
# https://community.plotly.com/t/redundant-legend-labels-unreproducable-tutorial/55175/2

!pip install plotly --upgrade

In [ ]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Google BigQuery Setup

#Project ID
pjt_id = "brilliant-era-321603" 

# data locations
bpd = "bigquery-public-data"
dataset = "iowa_liquor_sales"
table = "sales"

# data location full path
dataloc = f"`{bpd}.{dataset}.{table}`"

In [ ]:
# query: count table rows and shows years
row_count = pd.io.gbq.read_gbq(
    f"""
 SELECT
    COUNT(*) as total_rows
  FROM {dataloc}""",
    project_id=pjt_id,
)

yrs = pd.io.gbq.read_gbq(
    f"""SELECT
  DISTINCT(FORMAT_DATE('%Y', date)) AS years,
  FROM {dataloc}
  ORDER BY
    years DESC """,
    project_id=pjt_id,)

print(row_count)
yrs

   total_rows
0    19118960


,years
0,2020
1,2019
2,2018
3,2017
4,2016
5,2015
6,2014
7,2013
8,2012
9,None


In [ ]:
# query: top 5 sales by date 
sum_sales_date = pd.io.gbq.read_gbq(
    f"""
  SELECT
    date,
    FORMAT_DATE('%A', date) AS day_name,
    ROUND(SUM(sale_dollars),2) AS total_sales,
  FROM
    {dataloc}
  GROUP BY
    date
  ORDER BY
    total_sales DESC
  """,
    project_id=pjt_id,) 

sum_sales_date.head(5)

,date,day_name,total_sales
0,2013-10-04,Friday,3516318.05
1,2013-10-11,Friday,3278998.01
2,2020-09-29,Tuesday,2420417.11
3,2019-07-31,Wednesday,2357939.86
4,2019-09-05,Thursday,2290099.19


In [ ]:
# query: top 5sales by weekday name
sum_sales_wkday = pd.io.gbq.read_gbq(
    f"""
  SELECT
    FORMAT_DATE('%A', date) AS day_name,
    ROUND(SUM(sale_dollars),2) AS total_sales,
  FROM
    {dataloc}
  GROUP BY
    day_name
  ORDER BY
    total_sales DESC
  """,
    project_id=pjt_id,) 

sum_sales_wkday.head(5)

,day_name,total_sales
0,Monday,6.146334e+08
1,Wednesday,5.972763e+08
2,Tuesday,5.685008e+08
3,Thursday,5.415577e+08
4,Friday,2.400187e+08


In [ ]:
# query: top 5 sales by month name
sum_sales_mth = pd.io.gbq.read_gbq(
    f"""
  SELECT
    FORMAT_DATE('%B', date) AS month,
    ROUND(SUM(sale_dollars),2) AS total_sales,
  FROM
    {dataloc}
  GROUP BY
    month
  ORDER BY
    total_sales DESC
  """,
    project_id=pjt_id,) 

sum_sales_mth.head(5)

,month,total_sales
0,October,2.474805e+08
1,June,2.425365e+08
2,May,2.370718e+08
3,August,2.311448e+08
4,July,2.263198e+08


In [ ]:
# query: top 5sales by month (December)
dec_sum_sales = pd.io.gbq.read_gbq(
    f"""
  SELECT
    EXTRACT(DAY FROM date) AS the_day,
    ROUND(SUM(sale_dollars),2) AS total_sales,
  FROM
     {dataloc}
  WHERE
    FORMAT_DATE('%B', date) = 'December'
  GROUP BY
    the_day
  ORDER BY
    total_sales DESC
  """,
    project_id=pjt_id,) 

dec_sum_sales.head(5)

,the_day,total_sales
0,26,11284143.90
1,27,10255324.46
2,19,9622026.31
3,21,9506483.97
4,18,9504026.97


In [ ]:
# query: top 5 sales by month (June)
june_sum_sales = pd.io.gbq.read_gbq(
    f"""
  SELECT
    EXTRACT(DAY FROM date) AS the_day,
    ROUND(SUM(sale_dollars),2) AS total_sales,
  FROM
     {dataloc}
  WHERE
    FORMAT_DATE('%B', date) = 'June'
  GROUP BY
    the_day
  ORDER BY
    total_sales DESC
  """,
    project_id=pjt_id,) 

june_sum_sales.head(5)

,the_day,total_sales
0,26,12997654.91
1,27,11085004.80
2,25,10683658.61
3,5,9840296.04
4,4,9823520.72


In [ ]:
# query: top 5 sales by city
city_sum_sales = pd.io.gbq.read_gbq(
    f"""
  SELECT
    LOWER(city) AS city,
    ROUND(CAST(SUM(sale_dollars) AS numeric),0) AS total_sales,
  FROM
     {dataloc}
  GROUP BY
    city
  ORDER BY
    total_sales DESC
  """,
    project_id=pjt_id,) 

city_sum_sales.head(5)

,city,total_sales
0,des moines,316446115
1,cedar rapids,173729963
2,davenport,131458370
3,west des moines,98289070
4,waterloo,91743568


In [ ]:
# query: top 5 sales by county
county_sum_sales = pd.io.gbq.read_gbq(
    f"""
  SELECT
    LOWER(county) AS county,
    ROUND(CAST(SUM(sale_dollars) AS numeric),0) AS total_sales,
  FROM
     {dataloc}
  GROUP BY
    county
  ORDER BY
    total_sales DESC
  """,
    project_id=pjt_id,) 

county_sum_sales.head(5)

,county,total_sales
0,polk,577370868
1,linn,224323673
2,scott,184738695
3,johnson,153257799
4,black hawk,145297452


In [ ]:
# query: top 5 sales by vendor 

vendor_sum_sales = pd.io.gbq.read_gbq(
    f"""
  SELECT
    LOWER(vendor_name) AS vendor,
    ROUND(CAST(SUM(sale_dollars) AS numeric),0) AS total_sales
  FROM
    {dataloc}
  GROUP BY
    vendor
  ORDER BY
    total_sales DESC
  """,
    project_id=pjt_id,) 

vendor_sum_sales.head(5)

,vendor,total_sales
0,diageo americas,551505642
1,jim beam brands,199903939
2,sazerac company inc,115002985
3,pernod ricard usa,93186264
4,pernod ricard usa/austin nichols,92119352


In [ ]:
# query: top 5 sales by item 

item_sum_sales = pd.io.gbq.read_gbq(
    f"""
  SELECT
    LOWER(item_description) AS item,
    ROUND(CAST(SUM(sale_dollars) AS numeric),0) AS total_sales
  FROM
    {dataloc}
  GROUP BY
    item
  ORDER BY
    total_sales DESC
     """,
    project_id=pjt_id,) 

item_sum_sales.head(5)

,item,total_sales
0,black velvet,103941337
1,captain morgan spiced rum,72872932
2,titos handmade vodka,67739561
3,jack daniels old #7 black lbl,67262715
4,fireball cinnamon whiskey,55922339


In [ ]:
# query: top 5 sales by item & category

item_cat_sum_sales = pd.io.gbq.read_gbq(
    f"""
  SELECT
    LOWER(item_description) AS item,
    LOWER(category_name) AS category,
    ROUND(CAST(SUM(sale_dollars) AS numeric),0) AS total_sales
  FROM
    {dataloc}
  GROUP BY
    item,category
  ORDER BY
    total_sales DESC
       """,
    project_id=pjt_id,) 

item_cat_sum_sales.head(5)

,item,category,total_sales
0,black velvet,canadian whiskies,103941337
1,captain morgan spiced rum,spiced rum,72872932
2,jack daniels old #7 black lbl,tennessee whiskies,67262715
3,titos handmade vodka,american vodkas,60750450
4,fireball cinnamon whiskey,whiskey liqueur,55922339


In [ ]:
# query: top 5 sales by item & category for 2020

item_cat_sum_sales_2020 = pd.io.gbq.read_gbq(
    f"""
  SELECT
    FORMAT_DATE('%Y', date) AS year,
    LOWER(item_description) AS item,
    LOWER(category_name) AS category,
    ROUND(CAST(SUM(sale_dollars) AS numeric),0) AS total_sales
  FROM
    {dataloc}
  WHERE
    FORMAT_DATE('%Y', date) LIKE ('%2020%')
  GROUP BY
    item,
    category,
    year
  ORDER BY
    total_sales DESC
       """,
    project_id=pjt_id,) 

item_cat_sum_sales_2020.head(5)

,year,item,category,total_sales
0,2020,titos handmade vodka,american vodkas,20148376
1,2020,black velvet,canadian whiskies,11308314
2,2020,hennessy vs,imported brandies,9315170
3,2020,captain morgan original spiced,spiced rum,8696774
4,2020,crown royal,canadian whiskies,8448506


In [ ]:
# plot: top 5 sales by item & category for 2020
import plotly.express as px

item_2020 = (
    item_cat_sum_sales_2020.sort_values(by="total_sales", ascending=False)
    .head(10)
    .copy()
)
item_2020 = item_2020.sort_values(by="total_sales", ascending=True)

fig = px.bar(
    item_2020,
    x="total_sales",
    y="item",
    orientation="h",
    text="total_sales",
    color="category")

# update bar markers
fig.update_traces(textposition="outside")

# update plot details
fig.update_layout(
    {"plot_bgcolor": "rgba(255,255,255, 0.9)"},  # white background
    title={
        "text": f"2020 Sales by Item & Category ",
        "y": 0.98,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    },
    yaxis_title="" #remove axis titles
)

fig.show()

In [ ]:
# query: top 5 sales by item & category for past 10 years

item_cat_sum_sales = pd.io.gbq.read_gbq(
    f"""
  SELECT
    FORMAT_DATE('%Y', date) AS year,
    LOWER(item_description) AS item,
    LOWER(category_name) AS category,
    ROUND(CAST(SUM(sale_dollars) AS numeric),0) AS total_sales
  FROM
    {dataloc}
  GROUP BY
    item,
    category,
    year
  ORDER BY
    total_sales DESC
       """,
    project_id=pjt_id,) 

item_cat_sum_sales.sample()

,year,item,category,total_sales
0,2020,titos handmade vodka,american vodkas,20148376
1,2018,titos handmade vodka,american vodkas,14943390
2,2017,black velvet,canadian whiskies,13258183
3,2018,black velvet,canadian whiskies,13079561
4,2016,black velvet,canadian whiskies,12378453


In [ ]:
# create new table of items sales grouped by item,category, year 

item_sales = pd.io.gbq.read_gbq(
    f"""
CREATE TABLE
  `brilliant-era-321603.wbie.sale_items_by_year` AS
SELECT
  FORMAT_DATE('%Y', date) AS year,
  LOWER(item_description) AS item,
  LOWER(category_name) AS category,
  ROUND(CAST(SUM(sale_dollars) AS numeric),0) AS total_sales
FROM
    {dataloc}
GROUP BY
  item,
  category,
  year;""",
    project_id=pjt_id,) 

# run only once
# item_sales

In [ ]:
# Top 5 items by year (over 10 years)

item_sales_yrs = pd.io.gbq.read_gbq(
    f"""
SELECT   
  year,
  ARRAY_AGG(STRUCT(item,
      total_sales)
  ORDER BY
    total_sales DESC
  LIMIT
    5) AS top_sales
FROM `brilliant-era-321603.wbie.sale_items_by_year`
GROUP BY
  year
ORDER BY
  year DESC""",
    project_id=pjt_id,) 

item_sales_yrs.head(10)

,year,top_sales
0,2020,"[{'item': 'titos handmade vodka', 'total_sales..."
1,2019,"[{'item': 'titos handmade vodka', 'total_sales..."
2,2018,"[{'item': 'titos handmade vodka', 'total_sales..."
3,2017,"[{'item': 'black velvet', 'total_sales': 13258..."
4,2016,"[{'item': 'black velvet', 'total_sales': 12378..."
5,2015,"[{'item': 'black velvet', 'total_sales': 12113..."
6,2014,"[{'item': 'black velvet', 'total_sales': 11697..."
7,2013,"[{'item': 'black velvet', 'total_sales': 11096..."
8,2012,"[{'item': 'black velvet', 'total_sales': 10530..."
9,None,"[{'item': None, 'total_sales': None}]"


In [ ]:
# convert sql ARRAY_AGG(STRUCT) to df

yr_2020 = pd.DataFrame(item_sales_yrs.iloc[0, 1])
yr_2019 = pd.DataFrame(item_sales_yrs.iloc[1, 1])
yr_2018 = pd.DataFrame(item_sales_yrs.iloc[2, 1])
yr_2017 = pd.DataFrame(item_sales_yrs.iloc[3, 1])
yr_2016 = pd.DataFrame(item_sales_yrs.iloc[4, 1])
yr_2015 = pd.DataFrame(item_sales_yrs.iloc[5, 1])
yr_2014 = pd.DataFrame(item_sales_yrs.iloc[6, 1])
yr_2013 = pd.DataFrame(item_sales_yrs.iloc[7, 1])
yr_2012 = pd.DataFrame(item_sales_yrs.iloc[8, 1])


yr_2020["year"] = 2020
yr_2019["year"] = 2019
yr_2018["year"] = 2018
yr_2017["year"] = 2017
yr_2016["year"] = 2016
yr_2015["year"] = 2015
yr_2014["year"] = 2014
yr_2013["year"] = 2013
yr_2012["year"] = 2012


df = yr_2020.append(
    [yr_2019, yr_2018, yr_2017, yr_2016, yr_2015, yr_2014, yr_2013, yr_2012], sort=False
).reset_index(drop=True)

# References 

Google Cloud Docs: [SQL Date Formating](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators#format_date)